<div style="display:block">
    <div style="width: 20%; display: inline-block; text-align: left;">
        <img src="http://upload.wikimedia.org/wikipedia/en/0/0c/Mu_Sigma_Logo.jpg" style="height:75px; margin-left:0px" />
    </div>
    <div style="width: 59%; display: inline-block">
        <h1  style="text-align: center"> Social Media Analytics - Treatment pattern Identification </h1><br>
        <div style="width: 100%; text-align: center; display: inline-block;"><i>Author:</i> <strong>Varun Behl</strong> </div>
    </div>
    <div style="width: 20%; text-align: right; display: inline-block;">
        <div style="width: 100%; text-align: left; display: inline-block;">
            <i>Created: </i>
            <time datetime="2018-02-14" pubdate>13th Feb 2018</time>
            <i>Modified: </i>
            <time datetime="2018-02-21" pubdate>21st Feb 2018</time>
        </div>
    </div>
</div>

# Objective

The aim of this study is to identify the treatment pattern of patients (suffering or cured) from melanoma, based on the data scraped through different forums related to melanoma.The scraped data consists of comments of patients and their friends and family.

### Phase 1:
The first step is to predict whether the post discusses systemic therapy or not. This is done with a SVM clasification algorithm, which is most appropriate for text classification. 


In [1]:
# Importing necassary packages 
import string
import pandas as pd
import numpy as np
import nltk
import re

# Sklearn package to import necessary vectorizing algorithms
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer

#NLTK package for text mining operations 
from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk.tokenize import sent_tokenize
from nltk import pos_tag

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report as clsr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
import pickle
from sklearn.cross_validation import KFold
from nltk.tokenize import sent_tokenize
from __future__ import division
from scipy import sparse , vstack , hstack
from sklearn.svm import SVC
from sklearn.utils import check_random_state, safe_indexing

C:\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Step 1 : Importing social media dataset

##### Step 1a : Data scraped and tokenized to sentence level
The imported data set consists of posts which have been tokenized to sentence level with a corresponding column having flags 1 and 0: 1 (talking about drug consumption), and 0 (no mention of drug or treatment)

In [2]:
# Reading the sentenced tokenized dataset
posts = pd.read_csv('SMA_train_dataset.csv')

In [23]:
posts_test = pd.read_csv('DF_100_user_v1_5-16-2018.csv')

In [3]:

posts.head(5)


Tokenized.Sentences           date  Tag
0  I finished my courses (x4) of Ipilimumab, went...       2/8/2013    1
1  I have really struggled with the side effects ...     12/24/2012    1
2  I am just so stresses out trying to decide if ...      7/24/2011    1
3  My Ipilimumab treatment was before the stuff a...  5/5/2016 0:00    1
4  If it is decided that the combination is bette...  2/9/2016 0:00    0

### Step 2 :  Pre-processing text columns

Textual data has to be processed before incorporating it into the model. Various text mining operations are done on the textual data - like cleaning html tags, special characters, stripping URL tags, etc. Word tokens are created at the end of this process, which will be the independent features for the model. 


In [104]:
## Pre-processing the text column

#lemmatizer = WordNetLemmatizer()

def identity(arg):
    """
    Simple identity function works as a passthrough.
    """
    return arg

## Removing HTML tags
def cleanhtml(raw_html):
    cleanr = re.compile(r'<[^>]+>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

## Removing URL
def strip_URL(text):
    return re.sub(r'(https|http)?:\/\/(\w|\.|-|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)

## Removing Hashtags and '@' tags 
def strip_all_tags(text):
    entity_prefixes = ['@','#','[]']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

## Removing special characters 
def strip_special_chars(text):
    return re.sub('\W+',' ', text)

## Removing extra white spaces
def strip_extra_white_spaces(text):
    return re.sub(' +',' ',text).strip()

## Removing extra new lines
def strip_extra_new_lines(text):
    return re.sub("\n+","\n", text)


def tokenize(document):
    document = cleanhtml(document)
    document = strip_URL(document)
    document = strip_all_tags(document)
    document = strip_special_chars(document)
    document = strip_extra_white_spaces(document)
    document = strip_extra_new_lines(document)
    stopwords_custom = pd.read_csv("stopwords1.csv")
    stopwords_custom = stopwords_custom.iloc[:,0].map(lambda x:x.lower())
    #Breaking the document into sentences
    for sent in sent_tokenize(document):
        # Breaking the sentence into parts of speech-tagged tokens
        for token, tag in pos_tag(wordpunct_tokenize(sent)):
            # Applying pre-processing to the token
            token = token.lower() 
            #print token
            #If there is a stopword, ignore the token and continue
            if token.lower in stopwords_custom.tolist():
                continue

            # If there is a punctuation, ignore the token and continue
            if all(char in string.punctuation for char in token):
                continue
            
            yield token

#Lemmatize the token and yield
            #lemma = lemmatize(token, tag)
            #yield lemma

def lemmatize(token, tag):
    tag = {
       'N': wn.NOUN,
       'V': wn.VERB,
       'R': wn.ADV,
       'J': wn.ADJ
    }.get(tag[0], wn.NOUN)

    #return lemmatizer.lemmatize(token, tag)
    
def preprocessing(X):
    #tokens_dict = {}
    tokens_list = []
    count = 0
    for doc in X:
        count +=1
        print count
        tokens = list(tokenize(doc))
        # tokens_list
        tokens_list.append(tokens)
        
#tokens_dict[count] = inverse_transform(tokens) count += 1
    return tokens_list


#### Step 2a : Pre-processing and creating a list of word tokens

In [48]:
#pre-processed data :List of words (non-string)
words= preprocessing(posts['Tokenized.Sentences'])
#.map(lambda x:preprocessing(x))
#output type is a list of words ,here words are not in string format


i
finished
my
courses
x4
of
ipilimumab
went
off
pembrolizumab
and
unfortunately
have
had
signifcant
growth
i
have
really
struggled
with
the
side
effects
of
vemurafenib
but
here
are
some
over
the
counter
remedies
i
ve
had
some
success
with
i
am
just
so
stresses
out
trying
to
decide
if
i
should
go
for
the
aldesleukin
first
or
just
go
for
the
ipilimumab
my
ipilimumab
treatment
was
before
the
stuff
about
the
bifidobacteria
hit
the
news
if
it
is
decided
that
the
combination
is
better
i
think
that
will
become
the
first
immunotherapy
offered
but
only
to
paients
who
have
not
tried
ipilimumab
it
might
be
the
same
old
pain
or
it
might
be
an
ipilimumab
side
effect
or
both
he
had
aldesleukin
3
rounds
to
surgeries
lung
and
intestine
there
is
clear
evidence
that
interferon
does
have
benefits
for
3a
patients
i
m
stage
3a
and
decided
against
interferon
whether
you
are
light
sensitive
or
not
always
always
wear
sunglasses
while
taking
vemurafenib
just
to
be
sure
ipilimumab
ipilimumab
was
fda
approved
in

KeyboardInterrupt: 

In [71]:
posts_test['Tokenized.Sentences']

0       On Jan 21, 2016 5:40 PM Shaney2 wrote: Hi Ever...
1       My husband was diagnosed last May with Melanom...
2       The tumor was removed as well as the nearest L...
3       Because of the size and depth of the tumor, mo...
4                          5 weeks of radiation followed.
5       Shortly after radiation was completed, 2 new t...
6       Then again a very short time later, a cluster ...
7       It is the opinion of all the doctors involved ...
8       Next week he will under go a P.E.T scan (he ha...
9       On a side note, he also has Prostate Cancer, d...
10      The biopsy of the lymph nodes in his neck was ...
11      So, at this point we wait and see what the P.E...
12      I know each of you have your own problems and ...
13      Thanks for letting me vent!Sending lots of pra...
14      My mom was just diagnosed with metastic melano...
15      On Feb 15, 2016 1:46 AM Shaney2 wrote: On Feb ...
16      My husband was diagnosed last May with Melanom...
17      The tu

In [85]:
y = []
for i in x :
    y.append(tokenize(i))

In [97]:
x = posts_test['Tokenized.Sentences'][1]

In [101]:
preprocessing(x)

My husband was diagnosed last May with Melanoma on the side of his face, under his sideburn.


[<generator object tokenize at 0x0000000017C0DDC8>]

In [105]:
Words_test = preprocessing(posts_test['Tokenized.Sentences'])

1
On Jan 21 2016 5 40 PM Shaney2 wrote Hi Everyone I just signed up and I can see I have a lot of reading to do
2
My husband was diagnosed last May with Melanoma on the side of his face under his sideburn
3
The tumor was removed as well as the nearest Lymph Node and all tested positive
4
Because of the size and depth of the tumor more surgery was done to remove all lymph nodes on that side of his face as well as the sweat glands and saliva glands All tested positive the surgeon said he removed everything in the area that was not essential to life just to be on the safe side
5
5 weeks of radiation followed
6
Shortly after radiation was completed 2 new tumors appeared in the radiated area and were surgically removed
7
Then again a very short time later a cluster of small tumors have appeared in the same area as well as one large one on his chest
8
It is the opinion of all the doctors involved that further radiation and surgery are no longer options and they are now considering some type 

KeyboardInterrupt: 

In [28]:
Words_test

[[['o'],
  ['n'],
  [],
  ['j'],
  ['a'],
  ['n'],
  [],
  ['2'],
  ['1'],
  [],
  [],
  ['2'],
  ['0'],
  ['1'],
  ['6'],
  [],
  ['5'],
  [],
  ['4'],
  ['0'],
  [],
  ['p'],
  ['m'],
  [],
  ['s'],
  ['h'],
  ['a'],
  ['n'],
  ['e'],
  ['y'],
  ['2'],
  [],
  ['w'],
  ['r'],
  ['o'],
  ['t'],
  ['e'],
  [],
  [],
  ['h'],
  ['i'],
  [],
  ['e'],
  ['v'],
  ['e'],
  ['r'],
  ['y'],
  ['o'],
  ['n'],
  ['e'],
  [],
  [],
  ['i'],
  [],
  ['j'],
  ['u'],
  ['s'],
  ['t'],
  [],
  ['s'],
  ['i'],
  ['g'],
  ['n'],
  ['e'],
  ['d'],
  [],
  ['u'],
  ['p'],
  [],
  ['a'],
  ['n'],
  ['d'],
  [],
  ['i'],
  [],
  ['c'],
  ['a'],
  ['n'],
  [],
  ['s'],
  ['e'],
  ['e'],
  [],
  ['i'],
  [],
  ['h'],
  ['a'],
  ['v'],
  ['e'],
  [],
  ['a'],
  [],
  ['l'],
  ['o'],
  ['t'],
  [],
  ['o'],
  ['f'],
  [],
  ['r'],
  ['e'],
  ['a'],
  ['d'],
  ['i'],
  ['n'],
  ['g'],
  [],
  ['t'],
  ['o'],
  [],
  ['d'],
  ['o'],
  []],
 [['m'],
  ['y'],
  [],
  ['h'],
  ['u'],
  ['s'],
  ['b'],
  ['a'],
  [

In [7]:
# Word Tokens are created after preprocessing of the textual data
print words[0]
print type(words)

['i', 'finished', 'my', 'courses', 'x4', 'of', 'ipilimumab', 'went', 'off', 'pembrolizumab', 'and', 'unfortunately', 'have', 'had', 'signifcant', 'growth']
<type 'list'>


#### Step 2b : Creating a list of strings
For further operation with tfidf, input should be a string of words/phrase. The following code will create strings of phrases which can be appended to a list


In [7]:
#List of strings -in phrases -ready 
List_strings =[] 
for i in words:
    a = " ".join(str(x) for x in i)
    List_strings.append(a) 

In [17]:
len(List_strings)

5058

In [16]:
List_strings_test =[] 
for i in Words_test:
    a = " ".join(str(x) for x in i)
    List_strings_test.append(a)

In [17]:
print List_strings[0]
print List_strings[7]

i finished my courses x4 of ipilimumab went off pembrolizumab and unfortunately have had signifcant growth
there is clear evidence that interferon does have benefits for 3a patients


In [60]:
posts['Posts']= List_strings
posts.to_csv("Corrected_3.csv")

## Step 3 : Training and testing split
For classification model , the required training and testing dataset can be prepared with tts function . Output for features and and data point is a sparse matrix , where as the dependent variable is numpy array


In [9]:
# Dependent variable is defined
y = posts["Tag"]
subsample_size = 1.0
len(y)

5058

In [10]:
from sklearn.cross_validation import train_test_split as tts
from scipy import sparse

#Training and testing split - with a split of 70:30
X_train, X_test, y_train, y_test = tts(List_strings,y, test_size=0.3, random_state=123)


## Step 4 : Feature Generation
### Vectorizing
For feature generation, we need to create a matrix of vectorized features. For converting a collection of raw documents to a matrix of features, we use TF-IDF vectorizer (Term Frequency – Inverse Document Frequency). It assigns scores/weights to each word, which is considered as a data point for the model.

<b>Term Frequency: This summarizes how often a given word appears within a document.</b><br>
<b>Inverse Document Frequency: This downscales words that appear a lot across documents.</b><br>
TF-IDF are word frequency scores that try to highlight words that are more interesting i.e. frequent in a document, but not across documents.

The Tfidf Vectorizer will tokenize documents, learn the vocabulary and inverse document frequency weightings, and allows to encode new documents.

The dependent variable in the dataset has skewness which has to be eliminated before passing it through the model. 
The appended list of strings is passed through tfidf-vectorizer which a sparse csr matrix  .The generated matrix are used as features for the dataset ,with values being the tfidf score


In [11]:
vectorizer = TfidfVectorizer(use_idf= True)
#vectorizer= CountVectorizer()
X_1 = vectorizer.fit_transform(X_train)

### Step 5 : Balancing using Synthetic Minority Over-sampling Technique

This technique is followed to avoid overfitting which occurs when exact replicas of minority instances are added to the main dataset. A subset of data is taken from the minority class as an example and then new synthetic similar instances are created. These synthetic instances are then added to the original dataset. The new dataset is used as a sample to train the classification models.

SMOTE is used to create a balanced dataset which can be passed through the best-fit classifier model. The random state or seed should be equal to the assigned for the selected model

Input can be a spicy sparse matrix or numpy array


In [12]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=123, ratio = 0.8)
x_res, y_res = sm.fit_sample(X_1,y_train)

C:\Anaconda2\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


### Running Linear SVM model 
Support vector mahcine is a supervised learning method used for classification, regression and outliers detection. It is memory efficient as it uses a subset of training points in the decision function (called support vectors) . It works good for both binary and multi-class classification . It is flexible as we can select custom kernel as per the training data

SVM is independent of dimensionality of feature space. SVM uses overfitting protection , hence it can handle large number of features which are there in text classifiers

Balanced training set is in csr matrix format, which is used as independent variables and dependent variable is an array with equal dimensions for rows to that of training dataset


In [12]:
# from sklearn.svm import SVC

classifier = SVC(kernel= 'linear' , C =1 )
model = classifier.fit(X_1 ,y_train) 
tfs_test = vectorizer.transform(X_test)
feature_test = tfs_test
y_pred = classifier.predict(feature_test)

print(clsr(y_test.tolist(), y_pred.tolist()))

             precision    recall  f1-score   support

          0       0.83      0.87      0.85       859
          1       0.82      0.76      0.79       659

avg / total       0.82      0.82      0.82      1518



In [18]:
tfs_test_1 = vectorizer.transform(List_strings_test)
y_pred_test = classifier.predict(tfs_test_1)
len(y_pred_test)

9907

In [19]:
y_pred_test = pd.DataFrame(y_pred_test)


In [20]:
posts_test['New_tag'] = y_pred_test

In [21]:
posts_test.to_csv("DF_100_users_new_tag_5-16-2018.csv" , index = False)

### Running Naive Bayes model

The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf .

In [24]:
classifier = MultinomialNB()
model = classifier.fit(X_train ,y_train) 
#tfs_test = vectorizer.transform(X_test)
feature_test = X_test

from scipy.sparse import hstack
# feature_test = hstack((tfs_test,TempCount_test)).toarray()

y_pred_NB = classifier.predict(feature_test)

print(clsr(y_test.tolist(), y_pred_NB.tolist()))

             precision    recall  f1-score   support

          0       0.77      0.93      0.84       766
          1       0.88      0.65      0.74       601

avg / total       0.82      0.80      0.80      1367



### Running KNearestNeighbour Model

kNN is powerful as it does not assume anything about the data, other than a distance measure can be calculated consistently between any two instances. It is non-parametric or non-linear as it does not assume a functional form.


In [25]:
classifier = KNeighborsClassifier()
model = classifier.fit(X_train ,y_train) 
#tfs_test = vectorizer.transform(X_test)
feature_test = X_test

from scipy.sparse import hstack
# feature_test = hstack((tfs_test,TempCount_test)).toarray()

y_pred_Knn = classifier.predict(feature_test)


print(clsr(y_test.tolist(), y_pred_Knn.tolist()))

             precision    recall  f1-score   support

          0       0.78      0.82      0.80       766
          1       0.75      0.71      0.73       601

avg / total       0.77      0.77      0.77      1367



### K-Fold Cross Validation 

Cross Validation is a very useful technique for assessing the performance of machine learning models. It helps in knowing how the machine learning model would generalize to an independent data set.

K-fold is also a  model validation technique for assessing the effectiveness of the model, whether it is overfitting or underfitting. It is mainly used in settings where the goal is prediction, and one wants to estimate how accurately a predictive model will perform in practice . It randomly splits the entire dataset into k-folds. For each k folds in your dataset ,test the model on k – 1 folds of the data set. 

In [29]:
from sklearn.model_selection import KFold 
kf = KFold(n_splits=4)
for train_index, test_index in kf.split(X_1):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_1[train_index], X_1[test_index]
    y_train, y_test = y[train_index], y[test_index]
    classifier = SVC(kernel='linear' ,C = 1)
    model = classifier.fit(X_train ,y_train)
    feature_test = X_test
    y_pred = classifier.predict(feature_test)
    print(clsr(y_test.tolist(), y_pred.tolist()))

             precision    recall  f1-score   support

          0       0.72      0.92      0.81       559
          1       0.89      0.66      0.76       580

avg / total       0.81      0.79      0.78      1139

             precision    recall  f1-score   support

          0       0.83      0.85      0.84       664
          1       0.78      0.76      0.77       475

avg / total       0.81      0.81      0.81      1139

             precision    recall  f1-score   support

          0       0.88      0.82      0.85       709
          1       0.73      0.81      0.77       429

avg / total       0.82      0.82      0.82      1138

             precision    recall  f1-score   support

          0       0.84      0.88      0.86       672
          1       0.81      0.76      0.78       466

avg / total       0.83      0.83      0.83      1138



In [152]:
Final_posts = pd.read_csv('Final_posts.csv')


## Checking post level accuracy
#### Step 1 : Creating training and testing splits

In [153]:
posts_sorted = posts.sort_values('Docs')
train = posts_sorted['Docs']<=1700
test = posts_sorted['Docs']>1700
training_se = posts_sorted[train]
test_se = posts_sorted[test]


#### Step 2 : Preprocessing and tokenizing into a list

In [154]:
words_se_level = preprocessing(training_se['Tokenized.Sentences'])

In [155]:
List_strings_se_level =[] 
for i in words_se_level:
    a = " ".join(str(x) for x in i)
    List_strings_se_level.append(a)


#### Step 3 : TF- IDF vectorizing

In [156]:
# Training data set creation
X_se_level = vectorizer.fit_transform(List_strings_se_level)
y_se_level = training_se['New_flag']

#### Step 4 : Balancing using SMOTE

In [157]:
sm_se = SMOTE(random_state=123,ratio = 1.0)
x_res_train, y_res_train = sm_se.fit_sample(X_se_level,y_se_level)


C:\Anaconda2\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


#### Step 5 : Preprocessing test dataset

In [158]:
#Testing data creating - Preprocessing and Y_test creation
words_se_level_test = preprocessing(test_se['Tokenized.Sentences'])

List_strings_se_level_test =[] 
for i in words_se_level_test:
    a = " ".join(str(x) for x in i)
    List_strings_se_level_test.append(a)

  

In [159]:
X_se_test = vectorizer.transform(List_strings_se_level_test)
y_se_test = test_se['New_flag']  


In [160]:
x_res_train

<10846x7402 sparse matrix of type '<type 'numpy.float64'>'
	with 242167 stored elements in Compressed Sparse Row format>

#### Step 6 : Running Linear SVM model

In [161]:
classifier = SVC(kernel='linear' ,C =1)
model = classifier.fit(x_res_train ,y_res_train)   
#tfs_test = vectorizer.transform(X_test)
feature_test = X_se_test
from scipy.sparse import hstack
# feature_test = hstack((tfs_test,TempCount_test)).toarray()
y_pred = classifier.predict(feature_test)

print(clsr(y_se_test.tolist(), y_pred.tolist()))

             precision    recall  f1-score   support

          0       0.96      0.92      0.94      1129
          1       0.59      0.77      0.66       167

avg / total       0.92      0.90      0.91      1296



#### Step 6 : Appending predicted tag to test dataset

In [162]:
#appending predicted to test data set
test_se["predictions"] = y_pred

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


#### Step 7 : Creating post level data set from sentence level predicted dataset

In [163]:
tested_post_level = test_se.groupby('unique_key')['predictions'].agg(max)
test_dict = {'unique_key':tested_post_level.index.tolist(),'Predicted':tested_post_level.values}
test_df = pd.DataFrame(test_dict)

#### Step 8 : Merging dataset to calculate post level accuracy

In [164]:
post_level = pd.merge(Final_posts,test_df[['Predicted','unique_key']] ,on='unique_key')

In [165]:
y_test = post_level['TAG']
y_pred = post_level['Predicted']
pred = sum(post_level['Predicted'])
tag = sum(post_level['TAG'])
Precison = pred/tag
print Precison

0.8623188405797102
